In [37]:
import os
import dotenv
import duckdb
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [2]:
envs = dotenv.dotenv_values()
conn_string = envs.get("DUCKDB_CON")
token = envs.get("DUCKDB_TOKEN")

conn_string_with_token = f"{conn_string}?motherduck_token={token}"

In [3]:
with duckdb.connect(conn_string_with_token) as conn:
    tables_df = conn.execute("SHOW TABLES;").fetchdf()
    
tables_df

,name
0,Student_Exams_View
1,Student_Exams_record
2,Student_Exams_record_clean
3,Student_Profile
4,Teachers_data
5,countries_language_expand
6,countries_tour_data


In [4]:
with duckdb.connect(conn_string_with_token) as conn:
    students_exam_df = conn.execute("SELECT * FROM Student_Exams_record;").fetchdf()
    students_profile_df = conn.execute("SELECT * FROM Student_Profile;").fetchdf()
    teachers_df = conn.execute("SELECT * FROM Teachers_data;").fetchdf()

In [5]:
students_exam_df

,SN,Name,Gender,Math_1st_CA,Math_2nd_CA,Math _3rd_CA,English_Ist_CA,English _2nd_CA,English _3rd_CA,National_Values_1st_CA,...,National_Values _3rd_CA,Basic_Science_Ist_CA,Basic_Science_2nd_CA,Basic_Science _3rd_CA,Business_studies_1st_CA,business_studies_2nd_CA,Business_Studies_3rd _CA,Basic_Technology_1stCA,Basic_Technology_2nd_CA,Basic_Technology_3rd_CA
0,1,Anna Ishaya,F,87,75,56,87,91,77,78,...,80,56,76,81,67,85,78,76,65,70
1,2,Joy Zakaria,F,60,85,84,90,87,87,87,...,69,65,64,76,64,82,76,74,56,80
2,3,Sunday Hannah Theresa,F,61,68,56,76,80,79,87,...,68,65,87,72,62,76,72,70,68,86
3,4,Sunday Gaambo,M,81,85,75,89,90,84,68,...,89,60,86,74,60,79,70,56,56,62
4,5,Moses Peace,F,70,75,76,90,88,79,76,...,78,64,82,79,64,76,68,80,63,86
5,6,Solomon Grace Zugwai,F,77,71,74,93,86,82,68,...,87,67,82,69,64,76,69,84,62,80
6,7,Ogbonna Shedrack Izuchukwu,M,90,76,72,94,89,89,90,...,87,64,76,72,67,72,67,82,64,75
7,8,Rabi'U Sudais Ali,M,82,72,74,87,89,89,92,...,85,70,72,62,76,70,76,76,75,74
8,9,Adamu Aziza,F,60,75,76,87,85,60,67,...,69,62,70,69,76,73,56,79,76,75
9,10,Habila Precious,F,89,74,68,86,65,75,89,...,77,70,81,72,75,75,67,69,56,73


In [6]:
students_profile_df

,Student Name,Age,Gender,Parental Education,Household Income,Learning Disabilities,Attendance Rate (%),Exam Scores (Average),Participation in Extracurricular Activities,Study Habits,Has Access to Learning Materials,Internet Access at Home,Engagement in Class,Emotional Well-Being,Peer Influence,Teacher-Student Interaction,Support from Parents (Good/Moderate/Poor)
0,Anna Ishaya,13,Female,Higher Institution,Low,No,,51,No,Good,Yes,No,High,Moderate,Positive,Frequent,Good
1,Joy Zakaria,14,Female,Higher Institution,Medium,No,,50,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good
2,Sunday Hannah Theresa,15,Female,Secondary,Low,No,,46,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good
3,Sunday Gaambo,14,Male,Primary,Low,No,,54,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good
4,Moses peace,15,Female,Higher Institution,Medium,No,,56,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good
5,Solomon Zugwai,12,Female,Higher Institution,Medium,No,,50,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good
6,ogbonna Shedrack,13,Male,Higher Institution,Medium,No,,46,Yes,Good,Yes,No,High,Good,Positive,Frequent,Moderate
7,rabi'u Sudais Ali,14,Male,Primary,Low,No,,49,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good
8,ADAMU Aziza,16,Female,Higher Institution,High,No,,48,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good
9,habila Precious,15,Female,Higher Institution,Medium,No,,50,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good


In [7]:
teachers_df

,Teacher ID,Teacher Name,Years of Experience,Qualification,Subject Taught,Class Size,Teaching Methods,Interaction Frequency,Engagement Level,Performance,Participation in Extracurriculars
0,617,Avaishong Zipporah Bala,33,B.Aed Crs,Crs,41,multiple,Frequent,Moderate,Good,Yes
1,597,James Emmanuel,33,Bsc Admin And Planning,Civic Education,41,multiple,Frequent,Moderate,Good,Yes
2,12302,Hassan Kwaja,25,B.Tech,Basic Technology,45,multiple,Frequent,High,Good,Yes
3,1799,Audi Keziah Yusuf,15,B.Ed Bsc,Phe,41,multiple,Frequent,High,Good,Yes
4,2662,Arung Christiana,15,Nce,Computer,50,multiple,Frequent,High,Good,Yes
5,25219,Habila Samuel,10,Bsc Ed,Basic Science,41,multiple,Frequent,High,Good,Yes
6,3635,Peter Alheri Abba,8,B.A,English,52,multiple,Frequent,High,Good,Yes
7,18470,Musa Justina,10,Nce,French,59,multiple,Frequent,High,Good,Yes
8,25338,Shekari Fatima,10,Nce,Hausa,45,multiple,Frequent,High,Good,Yes
9,6572,Dauda Diana,10,Nce,Basic Technology,52,multiple,Frequent,High,Good,Yes


### Data explanation
There are three datasets, each from a different table extracted from the DB.<br>
The datasets are:
* Student performance data
* Student profile data
* Teachers' data

Each will be explained with the data overview seem so far, more insights will be obtained later on in the notebook
#### Student performance data
This dataset includes names of 40 students from a school in Nigeria, their test scores in certain subjects looked into, it also informed us of the gender of the students

***Descision:*** The scores can be merged together to get the overall average score of each student, names data removed, as they are neither relevant to the analysis, nor to the model to be built. The gender column should remain, unless otherwise necessary.

#### Student profile data
This dataset includes student names, average scores, and gender (all can be found in the first data) with more columns that are beneficial to model building and EDA in general.

***Decision:*** Once it is ascertained that the average score here matches with the average score that is intended to be calculated from the first data, then, all the information in the first data can be ignored as they have been repeated in the second data with more relevant columns, even. Otherwise, it (the intended average) will still be calculated and added to the second dataset. This data will be used for model training, to predict students' performance based on their data.

#### Teachers' data
This dataset contains information about 20 teachers in the same school as the sample students, it has information regarding their names, years of experience, and many more relevant columns, good for insights to help the school improve students performance, but model training is not necessary.

***Decision:*** This data is good to find some valuable insights on what to improve on, but no model training is required.

#### EDA of Student performance data

In [8]:
students_exam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   SN                        40 non-null     object
 1   Name                      40 non-null     object
 2   Gender                    40 non-null     object
 3   Math_1st_CA               40 non-null     object
 4   Math_2nd_CA               40 non-null     object
 5   Math _3rd_CA              40 non-null     object
 6   English_Ist_CA            40 non-null     object
 7   English _2nd_CA           40 non-null     object
 8   English _3rd_CA           40 non-null     object
 9   National_Values_1st_CA    40 non-null     object
 10  National_Values _2nd_CA   40 non-null     object
 11  National_Values _3rd_CA   40 non-null     object
 12  Basic_Science_Ist_CA      40 non-null     object
 13  Basic_Science_2nd_CA      40 non-null     object
 14  Basic_Science _3rd_CA     40

In [9]:
# The dataframe is treating the numbers as object even though they are integers, hence the need for conversion
exam_cols = students_exam_df.columns.to_list()[3:]
students_exam_df[exam_cols] = students_exam_df[exam_cols].apply(pd.to_numeric)

In [10]:
# Checking again to confirm
students_exam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   SN                        40 non-null     object
 1   Name                      40 non-null     object
 2   Gender                    40 non-null     object
 3   Math_1st_CA               40 non-null     int64 
 4   Math_2nd_CA               40 non-null     int64 
 5   Math _3rd_CA              40 non-null     int64 
 6   English_Ist_CA            40 non-null     int64 
 7   English _2nd_CA           40 non-null     int64 
 8   English _3rd_CA           40 non-null     int64 
 9   National_Values_1st_CA    40 non-null     int64 
 10  National_Values _2nd_CA   40 non-null     int64 
 11  National_Values _3rd_CA   40 non-null     int64 
 12  Basic_Science_Ist_CA      40 non-null     int64 
 13  Basic_Science_2nd_CA      40 non-null     int64 
 14  Basic_Science _3rd_CA     40

It has not been confirmed that the columns are now numerical

In [11]:
# determining the student's average in all the CAs
students_exam_df["average"] = students_exam_df[exam_cols].mean(axis=1).apply(round) #this line finds average test score of each student, then round it up to the nearest whole number
students_exam_df

,SN,Name,Gender,Math_1st_CA,Math_2nd_CA,Math _3rd_CA,English_Ist_CA,English _2nd_CA,English _3rd_CA,National_Values_1st_CA,...,Basic_Science_Ist_CA,Basic_Science_2nd_CA,Basic_Science _3rd_CA,Business_studies_1st_CA,business_studies_2nd_CA,Business_Studies_3rd _CA,Basic_Technology_1stCA,Basic_Technology_2nd_CA,Basic_Technology_3rd_CA,average
0,1,Anna Ishaya,F,87,75,56,87,91,77,78,...,56,76,81,67,85,78,76,65,70,76
1,2,Joy Zakaria,F,60,85,84,90,87,87,87,...,65,64,76,64,82,76,74,56,80,76
2,3,Sunday Hannah Theresa,F,61,68,56,76,80,79,87,...,65,87,72,62,76,72,70,68,86,73
3,4,Sunday Gaambo,M,81,85,75,89,90,84,68,...,60,86,74,60,79,70,56,56,62,75
4,5,Moses Peace,F,70,75,76,90,88,79,76,...,64,82,79,64,76,68,80,63,86,77
5,6,Solomon Grace Zugwai,F,77,71,74,93,86,82,68,...,67,82,69,64,76,69,84,62,80,76
6,7,Ogbonna Shedrack Izuchukwu,M,90,76,72,94,89,89,90,...,64,76,72,67,72,67,82,64,75,78
7,8,Rabi'U Sudais Ali,M,82,72,74,87,89,89,92,...,70,72,62,76,70,76,76,75,74,78
8,9,Adamu Aziza,F,60,75,76,87,85,60,67,...,62,70,69,76,73,56,79,76,75,72
9,10,Habila Precious,F,89,74,68,86,65,75,89,...,70,81,72,75,75,67,69,56,73,74


In [12]:
# confirm that exam average is numerical
students_profile_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   Student Name                                 40 non-null     object
 1   Age                                          40 non-null     object
 2   Gender                                       40 non-null     object
 3   Parental Education                           40 non-null     object
 4   Household Income                             40 non-null     object
 5   Learning Disabilities                        40 non-null     object
 6   Attendance Rate (%)                          40 non-null     object
 7   Exam Scores (Average)                        40 non-null     object
 8   Participation in Extracurricular Activities  40 non-null     object
 9   Study Habits                                 40 non-null     object
 10  Has Access to Le

In [13]:
# comparing the two averages (in the first two tables)
test_average = students_exam_df["average"].to_list()
exam_average = students_profile_df["Exam Scores (Average)"].apply(pd.to_numeric).to_list()

# summing each time they are equal in each index (position representing each student), if they are the same all through, it will be 0, any other number means they are not the same.
diff_sum = sum(avg[0] != avg[1] for avg in zip(test_average, exam_average))
diff_sum

40

40, intresting, meaning, none of them is actually the same, and they are completely different averages, as their name suggests. As mentioned earlier, it will be added to the student profile dataset and the analysis can now begin. All these steps were to confirm we don't have any data left out based on assumption that it is being repeated.

In [14]:
students_profile_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   Student Name                                 40 non-null     object
 1   Age                                          40 non-null     object
 2   Gender                                       40 non-null     object
 3   Parental Education                           40 non-null     object
 4   Household Income                             40 non-null     object
 5   Learning Disabilities                        40 non-null     object
 6   Attendance Rate (%)                          40 non-null     object
 7   Exam Scores (Average)                        40 non-null     object
 8   Participation in Extracurricular Activities  40 non-null     object
 9   Study Habits                                 40 non-null     object
 10  Has Access to Le

Two columns, `Age` and `Exam Scores (Average)` needs to be converted to numeric data type for ease of work

In [15]:
cols_to_num = ["Age", "Exam Scores (Average)"]
students_profile_df[cols_to_num] = students_profile_df[cols_to_num].apply(pd.to_numeric)

students_profile_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   Student Name                                 40 non-null     object
 1   Age                                          40 non-null     int64 
 2   Gender                                       40 non-null     object
 3   Parental Education                           40 non-null     object
 4   Household Income                             40 non-null     object
 5   Learning Disabilities                        40 non-null     object
 6   Attendance Rate (%)                          40 non-null     object
 7   Exam Scores (Average)                        40 non-null     int64 
 8   Participation in Extracurricular Activities  40 non-null     object
 9   Study Habits                                 40 non-null     object
 10  Has Access to Le

Analysis can now continue

In [16]:
students_profile_df

,Student Name,Age,Gender,Parental Education,Household Income,Learning Disabilities,Attendance Rate (%),Exam Scores (Average),Participation in Extracurricular Activities,Study Habits,Has Access to Learning Materials,Internet Access at Home,Engagement in Class,Emotional Well-Being,Peer Influence,Teacher-Student Interaction,Support from Parents (Good/Moderate/Poor)
0,Anna Ishaya,13,Female,Higher Institution,Low,No,,51,No,Good,Yes,No,High,Moderate,Positive,Frequent,Good
1,Joy Zakaria,14,Female,Higher Institution,Medium,No,,50,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good
2,Sunday Hannah Theresa,15,Female,Secondary,Low,No,,46,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good
3,Sunday Gaambo,14,Male,Primary,Low,No,,54,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good
4,Moses peace,15,Female,Higher Institution,Medium,No,,56,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good
5,Solomon Zugwai,12,Female,Higher Institution,Medium,No,,50,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good
6,ogbonna Shedrack,13,Male,Higher Institution,Medium,No,,46,Yes,Good,Yes,No,High,Good,Positive,Frequent,Moderate
7,rabi'u Sudais Ali,14,Male,Primary,Low,No,,49,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good
8,ADAMU Aziza,16,Female,Higher Institution,High,No,,48,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good
9,habila Precious,15,Female,Higher Institution,Medium,No,,50,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good


In [17]:
# First, the test average from performance data will be added as a column
students_profile_df["Test (Average)"] = students_exam_df["average"]
students_profile_df

,Student Name,Age,Gender,Parental Education,Household Income,Learning Disabilities,Attendance Rate (%),Exam Scores (Average),Participation in Extracurricular Activities,Study Habits,Has Access to Learning Materials,Internet Access at Home,Engagement in Class,Emotional Well-Being,Peer Influence,Teacher-Student Interaction,Support from Parents (Good/Moderate/Poor),Test (Average)
0,Anna Ishaya,13,Female,Higher Institution,Low,No,,51,No,Good,Yes,No,High,Moderate,Positive,Frequent,Good,76
1,Joy Zakaria,14,Female,Higher Institution,Medium,No,,50,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,76
2,Sunday Hannah Theresa,15,Female,Secondary,Low,No,,46,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,73
3,Sunday Gaambo,14,Male,Primary,Low,No,,54,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,75
4,Moses peace,15,Female,Higher Institution,Medium,No,,56,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,77
5,Solomon Zugwai,12,Female,Higher Institution,Medium,No,,50,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good,76
6,ogbonna Shedrack,13,Male,Higher Institution,Medium,No,,46,Yes,Good,Yes,No,High,Good,Positive,Frequent,Moderate,78
7,rabi'u Sudais Ali,14,Male,Primary,Low,No,,49,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,78
8,ADAMU Aziza,16,Female,Higher Institution,High,No,,48,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good,72
9,habila Precious,15,Female,Higher Institution,Medium,No,,50,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,74


In [18]:
# The name column has no relevance to the analysis, therefore, it will be removed.
students_profile_df = students_profile_df.drop(columns=["Student Name"])
students_profile_df

,Age,Gender,Parental Education,Household Income,Learning Disabilities,Attendance Rate (%),Exam Scores (Average),Participation in Extracurricular Activities,Study Habits,Has Access to Learning Materials,Internet Access at Home,Engagement in Class,Emotional Well-Being,Peer Influence,Teacher-Student Interaction,Support from Parents (Good/Moderate/Poor),Test (Average)
0,13,Female,Higher Institution,Low,No,,51,No,Good,Yes,No,High,Moderate,Positive,Frequent,Good,76
1,14,Female,Higher Institution,Medium,No,,50,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,76
2,15,Female,Secondary,Low,No,,46,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,73
3,14,Male,Primary,Low,No,,54,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,75
4,15,Female,Higher Institution,Medium,No,,56,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,77
5,12,Female,Higher Institution,Medium,No,,50,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good,76
6,13,Male,Higher Institution,Medium,No,,46,Yes,Good,Yes,No,High,Good,Positive,Frequent,Moderate,78
7,14,Male,Primary,Low,No,,49,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,78
8,16,Female,Higher Institution,High,No,,48,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good,72
9,15,Female,Higher Institution,Medium,No,,50,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,74


In [19]:
# Next is to check the uniqueness of each of the categorical columns
cat_cols = students_profile_df.select_dtypes("object").columns
for cat_col in cat_cols:
    col_data = students_profile_df[cat_col]
    print(col_data.value_counts())
    print("\n\n========================\n\n")

Gender
Female    23
Male      17
Name: count, dtype: int64




Parental Education
Higher Institution    23
Primary               11
Secondary              6
Name: count, dtype: int64




Household Income
Medium    26
Low       13
High       1
Name: count, dtype: int64




Learning Disabilities 
No    40
Name: count, dtype: int64




Attendance Rate (%)
    40
Name: count, dtype: int64




Participation in Extracurricular Activities
Yes    39
No      1
Name: count, dtype: int64




Study Habits 
Moderate    28
Good        11
Poor         1
Name: count, dtype: int64




Has Access to Learning Materials
Yes    40
Name: count, dtype: int64




Internet Access at Home 
No     31
Yes     9
Name: count, dtype: int64




Engagement in Class 
moderate    23
High        13
low          4
Name: count, dtype: int64




Emotional Well-Being 
Good        29
Moderate    11
Name: count, dtype: int64




Peer Influence
Positive    20
Neutral     20
Name: count, dtype: int64




Teacher-Student Interact

column`Attendance Rate (%)` will be removed as it doesn't contain any data 

In [20]:
students_profile_df = students_profile_df.drop(students_profile_df.columns[5], axis=1) # problem coming from not being able to remove the column with its name
students_profile_df

,Age,Gender,Parental Education,Household Income,Learning Disabilities,Exam Scores (Average),Participation in Extracurricular Activities,Study Habits,Has Access to Learning Materials,Internet Access at Home,Engagement in Class,Emotional Well-Being,Peer Influence,Teacher-Student Interaction,Support from Parents (Good/Moderate/Poor),Test (Average)
0,13,Female,Higher Institution,Low,No,51,No,Good,Yes,No,High,Moderate,Positive,Frequent,Good,76
1,14,Female,Higher Institution,Medium,No,50,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,76
2,15,Female,Secondary,Low,No,46,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,73
3,14,Male,Primary,Low,No,54,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,75
4,15,Female,Higher Institution,Medium,No,56,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,77
5,12,Female,Higher Institution,Medium,No,50,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good,76
6,13,Male,Higher Institution,Medium,No,46,Yes,Good,Yes,No,High,Good,Positive,Frequent,Moderate,78
7,14,Male,Primary,Low,No,49,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,78
8,16,Female,Higher Institution,High,No,48,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good,72
9,15,Female,Higher Institution,Medium,No,50,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,74


The two scores (test and exam) can be added together to form a total score, forming a total score of 160, then converted to percentage.<br>
(exam + test)/160 * 100. The exam and test averages can then be merged to form a single average

In [21]:
students_profile_df["Average"] = ((students_profile_df["Exam Scores (Average)"] + students_profile_df["Test (Average)"])*(100/160)).apply(round)
students_profile_df = students_profile_df.drop(columns=["Exam Scores (Average)", "Test (Average)"])
students_profile_df

,Age,Gender,Parental Education,Household Income,Learning Disabilities,Participation in Extracurricular Activities,Study Habits,Has Access to Learning Materials,Internet Access at Home,Engagement in Class,Emotional Well-Being,Peer Influence,Teacher-Student Interaction,Support from Parents (Good/Moderate/Poor),Average
0,13,Female,Higher Institution,Low,No,No,Good,Yes,No,High,Moderate,Positive,Frequent,Good,79
1,14,Female,Higher Institution,Medium,No,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,79
2,15,Female,Secondary,Low,No,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,74
3,14,Male,Primary,Low,No,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,81
4,15,Female,Higher Institution,Medium,No,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,83
5,12,Female,Higher Institution,Medium,No,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good,79
6,13,Male,Higher Institution,Medium,No,Yes,Good,Yes,No,High,Good,Positive,Frequent,Moderate,78
7,14,Male,Primary,Low,No,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,79
8,16,Female,Higher Institution,High,No,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good,75
9,15,Female,Higher Institution,Medium,No,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,78


In [22]:
# next is to categorise the average scores into low, medium and high.

bins = [0, 49, 70, 100]
labels = ["Low", "Medium", "High"]

students_profile_df["performance"] = pd.cut(students_profile_df["Average"], bins=bins, labels=labels)
students_profile_df

,Age,Gender,Parental Education,Household Income,Learning Disabilities,Participation in Extracurricular Activities,Study Habits,Has Access to Learning Materials,Internet Access at Home,Engagement in Class,Emotional Well-Being,Peer Influence,Teacher-Student Interaction,Support from Parents (Good/Moderate/Poor),Average,performance
0,13,Female,Higher Institution,Low,No,No,Good,Yes,No,High,Moderate,Positive,Frequent,Good,79,High
1,14,Female,Higher Institution,Medium,No,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,79,High
2,15,Female,Secondary,Low,No,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,74,High
3,14,Male,Primary,Low,No,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,81,High
4,15,Female,Higher Institution,Medium,No,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,83,High
5,12,Female,Higher Institution,Medium,No,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good,79,High
6,13,Male,Higher Institution,Medium,No,Yes,Good,Yes,No,High,Good,Positive,Frequent,Moderate,78,High
7,14,Male,Primary,Low,No,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,79,High
8,16,Female,Higher Institution,High,No,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good,75,High
9,15,Female,Higher Institution,Medium,No,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,78,High


In [23]:
# Removing the Average column to avoid redundancy
students_profile_df = students_profile_df.drop("Average", axis=1)
students_profile_df

,Age,Gender,Parental Education,Household Income,Learning Disabilities,Participation in Extracurricular Activities,Study Habits,Has Access to Learning Materials,Internet Access at Home,Engagement in Class,Emotional Well-Being,Peer Influence,Teacher-Student Interaction,Support from Parents (Good/Moderate/Poor),performance
0,13,Female,Higher Institution,Low,No,No,Good,Yes,No,High,Moderate,Positive,Frequent,Good,High
1,14,Female,Higher Institution,Medium,No,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,High
2,15,Female,Secondary,Low,No,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,High
3,14,Male,Primary,Low,No,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,High
4,15,Female,Higher Institution,Medium,No,Yes,Good,Yes,No,High,Moderate,Positive,Frequent,Good,High
5,12,Female,Higher Institution,Medium,No,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good,High
6,13,Male,Higher Institution,Medium,No,Yes,Good,Yes,No,High,Good,Positive,Frequent,Moderate,High
7,14,Male,Primary,Low,No,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,High
8,16,Female,Higher Institution,High,No,Yes,Moderate,Yes,No,High,Good,Positive,Frequent,Good,High
9,15,Female,Higher Institution,Medium,No,Yes,Good,Yes,No,High,Good,Positive,Frequent,Good,High


With all these, the final data for model building is ready, yet to be preprocessed, but ready. The intention is to use the other columns to determine whether a particular student will have either of the three performances (low, medium and high)

## Model building

In [24]:
X = students_profile_df.drop("performance", axis=1)
y = students_profile_df["performance"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [25]:
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
X_train_enc = encoder.fit_transform(X_train.select_dtypes("object")) 
X_test_enc = encoder.transform(X_test.select_dtypes("object"))

X_train_enc = pd.concat([X_train["Age"].reset_index(drop=True) , pd.DataFrame(X_train_enc, columns=X_train.select_dtypes("object").columns)], axis=1)

X_test_enc = pd.concat([X_test["Age"].reset_index(drop=True), pd.DataFrame(X_test_enc, columns=X_test.select_dtypes("object").columns)], axis=1)

In [26]:
X_test_enc

,Age,Gender,Parental Education,Household Income,Learning Disabilities,Participation in Extracurricular Activities,Study Habits,Has Access to Learning Materials,Internet Access at Home,Engagement in Class,Emotional Well-Being,Peer Influence,Teacher-Student Interaction,Support from Parents (Good/Moderate/Poor)
0,14,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,12,1.0,0.0,2.0,0.0,1.0,-1.0,0.0,0.0,1.0,0.0,0.0,1.0,-1.0
2,14,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0
3,15,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,15,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
5,12,1.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6,14,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,14,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0
8,14,1.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0
9,15,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0


In [29]:
y_encoder = LabelEncoder()
y_train_enc = y_encoder.fit_transform(y_train)
y_test_enc = y_encoder.transform(y_test)
y_test_enc

array([0, 1, 0, 0, 1, 1, 0, 1, 1, 1])

In [38]:
class Model:
    def __init__(self, models):
        self.models = models

    def _run_models_and_predict(self):
        metrics = {}
        for name, model in self.models.items():
            print(f"Training for model {name}")
            model = model.fit(X_train_enc, y_train_enc)
            y_pred = model.predict(X_test_enc)
            
            report = classification_report(y_test_enc, y_pred)
            metrics[name] = report
        return metrics

    def get_report(self):
        metrics = self._run_models_and_predict()
        for name, metric in metrics.items():
            print(f"\nModel name: {name}\n")
            print(f"Classification report:\n{metric}")

In [32]:
log_reg = LogisticRegression(penalty='l2', C=1, solver='liblinear')
dec_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
rand_frst = RandomForestClassifier(n_estimators=10, max_depth=3, random_state=42)

models_schema = {
    "Logistic Regression" : log_reg,
    "Decision Tree" : dec_tree,
    "Random Forest" : rand_frst
}

In [39]:
Model(models=models_schema).get_report()

Training for model Logistic Regression
Training for model Decision Tree
Training for model Random Forest

Model name: Logistic Regression

Classification report:
              precision    recall  f1-score   support

           0       0.75      0.75      0.75         4
           1       0.83      0.83      0.83         6

    accuracy                           0.80        10
   macro avg       0.79      0.79      0.79        10
weighted avg       0.80      0.80      0.80        10


Model name: Decision Tree

Classification report:
              precision    recall  f1-score   support

           0       0.60      0.75      0.67         4
           1       0.80      0.67      0.73         6

    accuracy                           0.70        10
   macro avg       0.70      0.71      0.70        10
weighted avg       0.72      0.70      0.70        10


Model name: Random Forest

Classification report:
              precision    recall  f1-score   support

           0       0.60    

The logistic regression model looks to be performing better than other models from the accuracy point of view, It can be confirmed further that it is really good by cross-validating

In [42]:
for name, model in models_schema.items():
    print(f"Model name: {name}")
    scores = cross_val_score(model, X_train_enc, y_train_enc, cv=5)
    print("Cross-validation scores:", scores)
    print("Average cross-validation score:", scores.mean())
    print("\n")

Model name: Logistic Regression
Cross-validation scores: [1.         0.66666667 0.5        0.83333333 0.66666667]
Average cross-validation score: 0.7333333333333333


Model name: Decision Tree
Cross-validation scores: [0.66666667 0.66666667 0.5        0.83333333 0.66666667]
Average cross-validation score: 0.6666666666666666


Model name: Random Forest
Cross-validation scores: [0.66666667 0.66666667 0.5        0.83333333 0.5       ]
Average cross-validation score: 0.6333333333333333




Logistic regression model has the best performance here also, after performing cross validation. This is mostl likely due to the simplicity and linearity of model and the limited data available. Therefore, logistic regression model will be selected for deployment

In [45]:
# Serialising the needed objects to be used in the deployment environ: encoders and log reg model
import joblib
import os
cwd = os.getcwd()
os.mkdir("objects")
base = os.path.join(cwd, "objects")

joblib.dump(encoder, os.path.join(base, "OrdinalEncoder.pkl")) # for encoding user input
joblib.dump(y_encoder, os.path.join(base, "LabelEncoder.pkl")) # so that I can reverse-encode the output
joblib.dump(log_reg, os.path.join(base, "LogisticRegression.pkl"))

print("successful")

successful
